Singularityではスクラッチからイメージを作成することができます。
主に以下の3つの方法があります。


* Docker Hubで提供されているイメージから作成する方法
* レシピファイルから作成する方法
* Singularity Hubで提供されているイメージから作成する方法

主な利用シナリオとしては、手元のroot権限があるラップトップやワークステーション、サーバでSingularityのイメージを作成し、
作成されたイメージはファイルとしてscpなどで共有計算機(AAIC)へ転送し、イメージを用いてプログラムを実行する、
などがあります。

サブコマンドとしてbuildを使用します。

In [ ]:
singularity build --help

ここでは、まず、Docker Hubで提供されているnvidia/cudaのイメージからSingularityのイメージを作成してみます。
以下のように実行します。
ここで、container0.simgはコンテナのファイル名で予めファイルは存在せず任意の名前をつけます。
また、docker://はDocker Hubを使うための宣言、nvidia/cudaはレポジトリ名、cuda:8.0-cudnn6-develはタグ名になります。

In [ ]:
sudo singularity build container0.simg docker://nvidia/cuda:8.0-cudnn6-devel

同様に、Singularity Hubで提供されているイメージから作成する場合は以下のように実行します。
ここでは、Singularity Hubのvosh/hello-worldで提供されているイメージから作成してみます。

In [ ]:
sudo singularity build container1.simg shub://vsoch/hello-world

次に、レシピファイルからSingularityのイメージを作成してみます。
レシピファイルのフォーマットの詳細は[ここ](http://singularity.lbl.gov/docs-recipes)を参照してください。
以下に簡単なサンプルファイルがあります。ファイル名はDockerと異なり任意の名前をつけることができます。

In [1]:
cat share/Singularity

Bootstrap: shub
From: vsoch/hello-world

%runscript
    exec echo "The runscript is the containers default runtime command!"

%files
   /home/vanessa/Desktop/hello-kitty.txt        # copied to root of container
   /home/vanessa/Desktop/party_dinosaur.gif     /opt/the-party-dino.gif #

%environment
    VARIABLE=MEATBALLVALUE
    export VARIABLE

%labels
   AUTHOR vsochat@stanford.edu

%post
    apt-get update && apt-get -y install python3 git wget
    mkdir /data
    echo "The post section is where you can install, and configure your container."

その後、コンテナ名とレシピファイルを指定してビルドします。

In [ ]:
cd share
sudo singularity build container3.simg Singularity

これらの生成されたSingularityイメージはshell, exec, runなどのサブコマンドに--writableオプションをつけることで改変することができます。
(つまり、インタラクティブにapt-getなどすることができます。)

In [ ]:
sudo singularity shell --writable container0.simg

また、イメージファイルでなくSandbox(コンテナ内のディレクトリ)をそのままマウントすることもできます。

In [ ]:
sudo singularity build --sandbox container4  docker://nvidia/cuda:8.0-cudnn6-devel